In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


# **ANALISIS EXPLORATORIO**

Cargamos e inspeccionamos los datos

In [2]:
Ctrl_trillado = pd.read_excel('CC FT 17   Formato de Control de Calidad Café de Trillado (1).xlsx', header = 5)
Formato_tostion = pd.read_excel('CC FT 18  Formato de  Tostión (1).xlsx', header = 5)
Formato_Despachos = pd.read_excel('CC FT 21   Formato de Control de Despachos (1).xlsx', header = 5)

In [3]:
print('info de tablas:')
print(Ctrl_trillado.info(),Formato_tostion.info(),Formato_Despachos.info())

info de tablas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   FECHA                              80 non-null     object 
 1   LOTE                               76 non-null     object 
 2   DENOMINACIÓN/     MARCA            76 non-null     object 
 3   CANTIDAD                           76 non-null     float64
 4   %H                                 77 non-null     object 
 5   Unnamed: 5                         78 non-null     object 
 6   MALLAS                             77 non-null     object 
 7   Unnamed: 7                         77 non-null     object 
 8   VERIFICACIÓN FISICA CAFÉ TOSTADO   77 non-null     object 
 9   NOTAS DE CATACIÓN                  76 non-null     object 
 10  PUNTAJE                            76 non-null     object 
 11  Unnamed: 11                        77 non-nu

In [4]:
print(Ctrl_trillado.head(),Formato_tostion.head(),Formato_Despachos.head())

      FECHA     LOTE   DENOMINACIÓN/     MARCA  CANTIDAD    %H Unnamed: 5  \
0       NaN        NaN                     NaN       NaN   NaN        NaN   
1       NaN        NaN                     NaN       NaN     %       C/NC   
2  19-07-22  01-190722            Madre Laura      765.0  10.9         C    
3   19-7-22  09-190722           Tabi Natural      204.0  10.2          C   
4  19-07-22  10-190722              Don Mario      165.0  10.7          C   

  MALLAS Unnamed: 7 VERIFICACIÓN FISICA CAFÉ TOSTADO   \
0    NaN        NaN                               NaN   
1      #       C/NC                              C/NC   
2     14         C                                  C   
3     14          C                                 C   
4     14          C                                 C   

                                   NOTAS DE CATACIÓN PUNTAJE Unnamed: 11  \
0                                                NaN     NaN         NaN   
1                                         

In [5]:
print('columnas de las tablas: ')
print(Ctrl_trillado.columns,Formato_tostion.columns,Formato_Despachos.columns)

columnas de las tablas: 
Index(['FECHA', 'LOTE  ', 'DENOMINACIÓN/     MARCA', 'CANTIDAD', '%H',
       'Unnamed: 5', 'MALLAS', 'Unnamed: 7',
       'VERIFICACIÓN FISICA CAFÉ TOSTADO ', 'NOTAS DE CATACIÓN', 'PUNTAJE',
       'Unnamed: 11', 'LIBERACIÓN DE LOTE', 'RESPONSABLE '],
      dtype='object') Index(['Fecha ', 'Lote ', 'Origen', 'Variedad', 'Proceso', 'Beneficio ',
       'Peso en Verde', 'Merma ', 'Peso en Tostado ', 'Perfil ',
       'Temp. De inicio y final', 'Tiempo de tueste', 'Observaciones ',
       'Tostador'],
      dtype='object') Index(['# PEDIDO ', 'FECHA DE TUESTE ', 'FECHA DE EMPAQUE ', 'TIPO DE CAFÉ   ',
       'PRESENTACIÓN ', 'Unnamed: 5', 'CANTIDAD ', 'Unnamed: 7', 'Unnamed: 8',
       'CLIENTE ', 'RESPONSABLE DESPACHO ', 'VERIFICA '],
      dtype='object')


Identificacion y limpieza de columnas importantes

In [47]:
Calidad = Ctrl_trillado[['CANTIDAD','%H','MALLAS']].dropna()
Tostion = Formato_tostion[['Peso en Verde','Peso en Tostado ','Merma ']].dropna()
Despachos =Formato_Despachos[['CANTIDAD ']].dropna()

In [7]:
print(Calidad.info(),Tostion.info(),Despachos.info())

<class 'pandas.core.frame.DataFrame'>
Index: 76 entries, 2 to 77
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   CANTIDAD  76 non-null     float64
 1   %H        76 non-null     object 
 2   MALLAS    76 non-null     object 
dtypes: float64(1), object(2)
memory usage: 2.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Peso en Verde     499 non-null    float64
 1   Peso en Tostado   499 non-null    float64
 2   Merma             499 non-null    float64
dtypes: float64(3)
memory usage: 11.8 KB
<class 'pandas.core.frame.DataFrame'>
Index: 1583 entries, 1 to 1583
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CANTIDAD   1583 non-null   float64
dtypes: float64(1)
memory usage: 24.7 KB
None None None

In [8]:
print(Calidad.head())
print(Tostion.head())
print(Despachos.head())

   CANTIDAD    %H MALLAS
2    765.00  10.9     14
3    204.00  10.2     14
4    165.00  10.7     14
5      0.45  10.5     14
6    105.00  10.7     14
   Peso en Verde  Peso en Tostado      Merma 
0           9.00              7.65  15.000000
1           3.00              2.50  16.666667
2           9.00              7.52  16.444444
3          11.70              9.79  16.324786
4           0.45              0.34  24.444444
   CANTIDAD 
1        3.0
2        1.0
3        2.0
4        1.0
5        3.0


Homogenizacion de los datos (float64)

In [9]:
Calidad = Calidad.apply(lambda col: pd.to_numeric(col, errors='coerce') if col.dtypes == 'object' else col)
Tostion = Tostion.apply(lambda col: pd.to_numeric(col, errors='coerce') if col.dtypes == 'object' else col)
Despachos = Despachos.apply(lambda col: pd.to_numeric(col, errors='coerce') if col.dtypes == 'object' else col)

In [10]:
print(Calidad.info(),Tostion.info(),Despachos.info())

<class 'pandas.core.frame.DataFrame'>
Index: 76 entries, 2 to 77
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   CANTIDAD  76 non-null     float64
 1   %H        76 non-null     float64
 2   MALLAS    76 non-null     int64  
dtypes: float64(2), int64(1)
memory usage: 2.4 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Peso en Verde     499 non-null    float64
 1   Peso en Tostado   499 non-null    float64
 2   Merma             499 non-null    float64
dtypes: float64(3)
memory usage: 11.8 KB
<class 'pandas.core.frame.DataFrame'>
Index: 1583 entries, 1 to 1583
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CANTIDAD   1583 non-null   float64
dtypes: float64(1)
memory usage: 24.7 KB
None None None


# **Definición de variables y preprocesamiento**

se escoje la variable objetivo y las variables de enrtada

In [11]:
VariableObjetivoY = Ctrl_trillado[['PUNTAJE']]
VariableEntradaX = pd.concat([Calidad, Tostion], axis=1)


In [12]:
print(VariableObjetivoY.info())
print(VariableEntradaX.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PUNTAJE  76 non-null     object
dtypes: object(1)
memory usage: 892.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
Index: 499 entries, 2 to 498
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CANTIDAD          76 non-null     float64
 1   %H                76 non-null     float64
 2   MALLAS            76 non-null     float64
 3   Peso en Verde     499 non-null    float64
 4   Peso en Tostado   499 non-null    float64
 5   Merma             499 non-null    float64
dtypes: float64(6)
memory usage: 27.3 KB
None


vemos que hay una incongruencia con la cantidad de valores, asi que para reemplazar los NaN usaremos la Media

In [13]:
VariableEntradaX['CANTIDAD'] = VariableEntradaX['CANTIDAD'].fillna(VariableEntradaX['CANTIDAD'].mean())
VariableEntradaX['%H'] = VariableEntradaX['%H'].fillna(VariableEntradaX['%H'].mean())
VariableEntradaX['MALLAS'] = VariableEntradaX['MALLAS'].fillna(VariableEntradaX['MALLAS'].mean())

In [14]:
print(VariableObjetivoY.info())
print(VariableEntradaX.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PUNTAJE  76 non-null     object
dtypes: object(1)
memory usage: 892.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
Index: 499 entries, 2 to 498
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CANTIDAD          499 non-null    float64
 1   %H                499 non-null    float64
 2   MALLAS            499 non-null    float64
 3   Peso en Verde     499 non-null    float64
 4   Peso en Tostado   499 non-null    float64
 5   Merma             499 non-null    float64
dtypes: float64(6)
memory usage: 27.3 KB
None


Prosigo con el escalado


In [15]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(VariableEntradaX)
X_scaled = pd.DataFrame(X_scaled, columns=VariableEntradaX.columns)

In [26]:
print(X_scaled.head(20))
X = X_scaled[0:95].dropna()
print(X.info())

    CANTIDAD        %H    MALLAS  Peso en Verde  Peso en Tostado     Merma 
0   8.658851  3.847442 -0.499081      -0.311508         -0.310390 -0.312849
1   0.969189 -4.025016 -0.499081      -0.224248         -0.221477 -0.368406
2   0.434613  1.598168 -0.499081      -0.587830         -0.591619  3.401540
3  -1.820884 -0.651106 -0.499081      -0.587830         -0.591619  3.401540
4  -0.387811  1.598168 -0.499081       0.334861          0.356649 -0.823400
5  -0.387811  1.598168 -0.499081      -0.505418         -0.506624 -0.364438
6   1.051431  1.598168 -0.499081      -0.408463         -0.409094 -0.209671
7  -1.662567 -2.900379 -0.499081      -0.214552         -0.209335 -0.596587
8  -1.561135 -0.651106 -0.499081       2.112375          2.136854 -0.209671
9  -1.347305  2.722805 -0.499081      -0.214552         -0.213252 -0.209671
10 -0.867558  1.598168 -0.499081       0.755001          0.765959 -0.209671
11  6.191579  7.221352 -0.499081       0.338093          0.349207 -0.385180
12  1.051431

division de los datos en conjuntos de prueba y entrenamiento


In [46]:
x_train, x_test, y_train, y_test = train_test_split(X, VariableObjetivoY, train_size=0.80, test_size=0.20, random_state=42)

# **ENTRENAMIENTO DE MODELOS DE REGRESION**

Modelo #1

In [ ]:
LinearR = LinearRegression()
LinearR.fit(x_train, y_train)

Modelo #2


In [ ]:
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)